## Код преназначен для парсинга новостей с сайта администрации Республики Коми // The code is intended for parsing news from the website of the administration of the Komi Republic
Дата создания: 08.05.2022 // Date of creation: 08.05.2022

Автор: Шилова надежда // Author: Nadejda Shilova 

In [1]:
#Необходимые библиотеки // Required libraries
from bs4 import BeautifulSoup
from openpyxl import Workbook
from openpyxl.styles import Border, Side, Alignment, NamedStyle
import pandas as pd
from razdel import sentenize
import re
import requests
from time import sleep

In [2]:
def parsing_news(news_rus):
    """
    Функция для сбора новостей с сайта администрации. // Function for collecting news from the administration's website.
    Параметры // Parameters:
            news_rus: номер новости в ссылке на русском // the news number in the link in Russian
    Результат выполнения // Execution result:
            [list_rus, list_komi]: список, который содержит два списка новостей на русском и коми языках // the list contains two lists of news in Russian and Komi languages'''
    """
    
    #Ссылка на новость на русском // Link to the news in Russian
    url_rus = "https://rkomi.ru/news/" + str(news_rus)
    
    #Получение страницы на русском // Getting the page in Russian
    request_rus = session.get(url_rus)
    soup_rus = BeautifulSoup(request_rus.text)
    
    list_rus = []
    
    #Получение заголовка на русском // Getting the title in Russian
    for tag in soup_rus.find_all("title"):
        title = re.search(r'[^|]*', tag.text)
        list_rus.append(title.group(0))
    
    #Получение текста новости на русском // Getting the news text in Russian
    for tag in soup_rus.find_all("p"):
        list_rus.append(tag.text)
        
    
    #Редирект на коми язык // Redirect to Komi
    url_komi = "https://rkomi.ru/language/kv"
    
    #Получение страницы на коми // Getting the page om Komi
    request_komi = session.get(url_komi)
    soup_komi = BeautifulSoup(request_komi.text, 'lxml')
    
    list_komi = []
    
    #Получение заголовка на коми // Getting the title in Komi
    for tag in soup_komi.find_all("title"):
        title = re.search(r'[^|]*', tag.text)
        list_komi.append(title.group(0))
    
    #Получение текста новости на коми // Getting the news text in Komi
    for tag in soup_komi.find_all("p"):
        list_komi.append(tag.text)
    
    #Перерыв секунда, перестраховка, чтобы не заблокировали // Break a second
    sleep(1)
    
    return [list_rus, list_komi]

In [3]:
start_ind_news = 11 #Номер самой ранней новости для сбора данных // The number of the earliest news to collect data
end_ind_news = 10 #Номер самой поздней новости для сбора данных //  The oldest news number for data collection

In [4]:
#Запуск ссесии с браузером // Starting a session with the browser
session = requests.Session() 
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'
})

In [5]:
#Получение всех заданных новостей с сайта // Getting all the specified news from the site
news_list = []
while start_ind_news >= end_ind_news:
    news_list.append(parsing_news(start_ind_news))
    start_ind_news -= 1

In [290]:
def split_sentence(news):
    """
    Функция для разделения данных на предложения. // Function for dividing data into sentences. 
    Параметры // Parameters:
            news: новость в виде списка, включающая список на русском и список на коми языках // news in the form of a list, including a list in Russian and a list in Komi
    Результат выполнения // Execution result:
            [list_sentenize_komi, list_sentenize_rus]: список, который содержит два списка, разделённых на предложения новостей, на коми и русском языках // a list that contains two lists divided into sentences news, in Komi and Russian'''
    """
    list_sentenize_rus = []
    list_sentenize_komi = []
    for ind_lang, language_news in enumerate(news):
        #Новость на русском // News in Russian
        if ind_lang == 0:
            for ind, sentence in enumerate(language_news):
                #Заголовок // Title
                if ind == 0:
                    list_sentenize_rus.append(sentence)
                #Тело новости // News content
                elif sentence not in ('', '\n'):
                    #Разделение на предложения // Division into sentences
                    text_generator_rus = sentenize(sentence)
                    for text_sentenize in text_generator_rus:
                        list_sentenize_rus.append(text_sentenize.text)
                    del text_generator_rus
        #Новость на коми // News in Komi
        else:
            for ind, sentence in enumerate(language_news):
                #Заголовок // Title
                if ind == 0:
                    list_sentenize_komi.append(sentence)
                #Тело новости // News content
                elif sentence not in ('', '\n'):
                    #Разделение на предложения // Division into sentences
                    text_generator_komi = sentenize(sentence)
                    for text_one_sentenize in text_generator_komi:
                        list_sentenize_komi.append(text_one_sentenize.text)
                    del text_generator_komi
    return [list_sentenize_komi, list_sentenize_rus]

In [291]:
#Создание файла excel // Creating an excel
wb = Workbook()
ws = wb.active
#Присвоение имени листа // Assigning a sheet name
ws.title = "Коми-рус-англ"
#Добавление заголовков таблицы // Adding Table Headers
ws['A1'] = 'Коми язык'
ws['B1'] = 'Русский язык'

#Первая строчка заполнения данных // First row for the data
start_row = 2

#Создание стиля ячейки // Creating a cell style
style_table = NamedStyle(name="style_table")
style_table.alignment = Alignment(wrap_text=True) #Выравнивание // Alignment
thins = Side(border_style="thin", color="0000ff") #Тип линии // Type of line
style_table.border = Border(right=thins, left=thins, top=thins, bottom=thins) #Границы таблицы // Borders of table

In [292]:
number_check = 1 #Номер новости для доработки // News number for correction
for news in news_list:
    #Если новость существует на двух языках // If the news exists in two languages
    if news[0] != news[1]:
        processed_text = split_sentence(news)
        #Если количество предложений на двух языках совпадает // If the count of sentences in two language is the same
        if len(processed_text[0]) == len(processed_text[1]):
            #Добавление новости в общий excel // Adding news to excel
            for ind, text in enumerate(processed_text[0]):
                ws.cell(row=start_row, column=1, value=text).style = style_table
                ws.cell(row=start_row, column=2, value=processed_text[1][ind]).style = style_table
                start_row += 1
        else:
            #Создание файла excel для ручного анализа // Creating an excel for manual analysis
            wb_check = Workbook()
            ws_check = wb_check.active
            #Присвоение имени листа // Assigning a sheet name
            ws_check.title = "Коми-рус-англ"
            #Добавление заголовков таблицы // Adding Table Headers
            ws_check['A1'] = 'Коми язык'
            ws_check['B1'] = 'Русский язык'
            #Первая строчка заполнения данных // First row for the data
            start_row_check = 2
            #Заполенение excel коми предложениями // Filling out excel with Komi sentences
            for text in processed_text[0]:
                ws_check.cell(row=start_row_check, column=1, value=text).style = style_table
                start_row_check += 1
            #Первая строчка заполнения данных // First row for the data
            start_row_check = 2
            #Заполенение excel русскими предложениями Filling out excel with Russian sentences
            for text in processed_text[1]:
                ws_check.cell(row=start_row_check, column=2, value=text).style = style_table
                start_row_check += 1
            #Сохранение файла для ручного анализа // Save the file for manual analysis
            wb_check.save(str(number_check) + '.xlsx')
            number_check += 1
#Сохранение обработанного файла // Saving the processed file
wb.save('parsing_administration.xlsx')